# Пробуем разнообразить способ с Вебинара №1

In [8]:
import json
import re
from pprint import pprint

import numpy as np
import pandas as pd
from IPython.core.display import HTML, display
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neural_network import MLPClassifier

In [40]:
train = pd.read_csv("../Data/train_fd.csv", index_col="index")

x_train = train["full_description"]
y_train = train["target"]

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x_train)

In [42]:
X_train_counts.shape

(15650, 18279)

In [43]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(15650, 18279)

In [44]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(15650, 18279)

In [128]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='modified_huber', penalty='elasticnet', l1_ratio=0.2,
                         alpha=1e-4, random_state=43,
                         max_iter=5, tol=None))
])

In [129]:
text_clf = text_clf.fit(x_train, y_train)
text_clf.score(x_train, y_train)

0.9847284345047923

In [130]:
X_test = pd.read_csv("../Data/test_fd.csv")
y_pred = text_clf.predict(X_test["full_description"])

In [131]:
y_pred

array([8322, 3323, 5223, ..., 5223, 3119, 4323], dtype=int64)

In [132]:
submission = X_test[["index"]].assign(target=y_pred)
display(submission.head(4))
submission.to_csv("../Results/submission_sk_v15.csv", index=False)

index  target
0  26461447    8322
1  26464220    3323
2  26467473    5223
3  26468989    4323